In [2]:
from comet_ml import API
import comet_ml
import io
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import defaultdict
import time


from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from colour import Color



In [3]:
comet_api = API(api_key='YOUR KEY')
comet_api.get()

['ketrint']

In [4]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']


In [5]:
def get_parameter_by_key(exp, key):
    parameters = exp.get_parameters_summary()
    for parameter in parameters:
        if parameter['name'] == key:
            return parameter['valueCurrent']
    return None

def new_to_old_metric(exp, key):
    metric = exp.get_metrics(key)
    vals = [float(m['metricValue']) for m in metric]
    return vals

def stack_lists(data, n=1000):
    new_data = []
    for d in data:
        if len(d) > n:
            new_data.append(d[:n])
        elif len(d) < n:
            new_data.append(
                np.concatenate([d, d[-1].repeat(n - len(d))])
            )
    return np.vstack(new_data).T

def preprocess_gp(vals):
    return np.minimum.accumulate(vals)

def add_zero_point(vals, point):
    return np.array([point] + vals.tolist())

In [25]:
experiments_names = ['val-clustering', 'val-clustering-2',
                     'val-clustering-3']

In [72]:
experiments = []
for i in range(len(experiments_names)):
    experiments.append(comet_api.get(workspace='ketrint', project_name=experiments_names[i]))

In [73]:
experiments = [item for sublist in experiments for item in sublist]

In [261]:
from collections import defaultdict

good_showers = defaultdict(list)
stuck_showers = defaultdict(list)
broken_showers = defaultdict(list)
lost_showers = defaultdict(list)
x = defaultdict(list)
y = defaultdict(list)
z = defaultdict(list)
tx = defaultdict(list)
ty = defaultdict(list)

for exp in experiments:
    if exp.get_command()[2] == '../data/rand_preprocessed.pt':
        key = exp.get_command()[-1].split('/')[-1].split('_')[2].split('.')[0]
        if key == 'HDBBSCAN':
            key = 'HDBSCAN'
        good_showers[key].append(new_to_old_metric(exp, "Good showers")[0])
        stuck_showers[key].append(new_to_old_metric(exp, "Stuck showers"))
        broken_showers[key].append(new_to_old_metric(exp, "Broken showers"))
        lost_showers[key].append(new_to_old_metric(exp, "Lost showers"))
        x[key].append(new_to_old_metric(exp, "MAE for x"))
        y[key].append(new_to_old_metric(exp,  "MAE for y"))
        z[key].append(new_to_old_metric(exp, "MAE for z"))
        tx[key].append(new_to_old_metric(exp, "MAE for tx"))
        ty[key].append(new_to_old_metric(exp, "MAE for ty"))

In [262]:
dict_list = [good_showers, stuck_showers, 
                   broken_showers, lost_showers, 
                   x, y, z, tx, ty]

In [263]:
dict_names = ['good_showers', 'stuck_showers', 
                   'broken_showers', 'lost_showers', 
                   'MAE for x', 'MAE for y', 'MAE for z', 'MAE for tx', 'MAE for ty']

In [264]:
#fix problems with spilling the keys names 

for dictionary in dict_list:
    dictionary['edge']. append(dictionary['RAND'][1])
    del dictionary['RAND'][1]

In [265]:
for dictionary in dict_list:
    for key in dictionary: 
        dictionary[key] = [np.round(np.mean(dictionary[key])*100,2), 
                           np.round(np.std(dictionary[key]/np.mean(dictionary[key]))*100, 4)]

In [266]:
df = pd.DataFrame.from_dict(dict_list)

In [267]:
df = df.assign(Metric = dict_names)

In [268]:
df = df.set_index('Metric') 

In [269]:
df

,HDBSCAN,edge,RAND,em
Metric,,,,
good_showers,"[71.45, 7.3211]","[77.83, 1.5734]","[82.52, 0.8876]","[82.15, 0.8735]"
stuck_showers,"[19.49, 33.6242]","[13.09, 10.9267]","[8.42, 22.2931]","[5.72, 10.0036]"
broken_showers,"[2.06, 24.0093]","[1.02, 4.3849]","[1.2, 14.0249]","[1.16, 11.0564]"
lost_showers,"[7.01, 27.3026]","[8.06, 15.8995]","[7.87, 20.465]","[10.98, 3.2215]"
MAE for x,"[30.8, 0.7487]","[30.98, 0.5454]","[31.0, 0.3728]","[31.04, 0.439]"
MAE for y,"[24.26, 0.5261]","[24.32, 0.4381]","[24.35, 0.6839]","[24.39, 0.5451]"
MAE for z,"[20.46, 1.4009]","[20.89, 0.4399]","[20.84, 0.7055]","[20.9, 0.63]"
MAE for tx,"[1.31, 12.6122]","[1.18, 2.5696]","[1.27, 8.5891]","[1.09, 3.8994]"
MAE for ty,"[1.25, 6.4473]","[1.18, 1.442]","[1.23, 6.0274]","[1.11, 2.6425]"
